In [1]:
import pandas as pd
# import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import xgboost as xgb
from xgboost import XGBClassifier, plot_importance
import warnings
from sklearn.metrics import precision_score,recall_score,roc_auc_score,roc_curve
import pickle

train = pd.read_csv('bert_trad_train.csv')

In [2]:
def from_data_x():
    data = [eval(str_list) for str_list in train['feature']]
    data = pd.DataFrame(data)
    return data.to_numpy()

def from_data_y():
    labels = train['labels']
    return labels.to_numpy()


# 假设data是包含数据的DataFrame，target是包含标签的Series
X = from_data_x()  # 特征数据
y = from_data_y()  # 标签数据

print(X.shape)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100,shuffle=True)

(4860, 790)


In [3]:
count = 0
for ele in  y_train:
    if ele == 1:
        count += 1

print('       Depressed     Non-depressed')
print('train:',count,'           ',X_train.shape[0] - count)
count = 0
for ele in  y_test:
    if ele == 1:
        count += 1
print('test :',count,'           ',X_test.shape[0] - count)
count = 0
for ele in y:
    if ele == 1:
        count += 1
print('total:',count,'           ',X.shape[0] - count)

       Depressed     Non-depressed
train: 674             3214
test : 156             816
total: 830             4030


In [4]:


def FP_TP_ids(model=None):
    if model == None:
        print('未设置模型')
        return None
    y_pred_all = model.predict(X)
    if len(y_pred_all) != len(y):
        print('标签不匹配')
        return None
    ids_list = []
    for i in range(len(y)):
        if y_pred_all[i] == 1:
            ids_list.append(i)

    return ids_list

ids_list =  []




def model_fit():
    """模型训练"""
    # XGBoost训练过程，下面的参数是调试出来的最佳参数组合
    model = XGBClassifier(n_estimators= 4,max_depth= 11, min_child_weight= 1,gamma= 0.3,colsample_bytree= 1,subsample=1,reg_alpha= 0.1, reg_lambda= 1,learning_rate=0.25)
    # model = RandomForestClassifier(n_estimators=200)

    # model = AdaBoostClassifier(random_state=37)
    model.fit(X_train, y_train)

    # 对验证集进行预测——类别
    y_pred = model.predict(X_test)
    # y_pred = model.predict(X_train)

	# 对验证集进行预测——概率
    y_pred_proba = model.predict_proba(X_test)
    # print(y_pred_proba)
    # 结果类别是1的概率
    y_pred_proba_ = []
    for i in y_pred_proba.tolist():
        exit(0)
        y_pred_proba_.append(i[1])
    # print('y_pred_proba：', y_pred_proba_)

    # 模型对验证集预测结果评分
    # metrics_sklearn(y_pred,y_test)

    metrics_sklearn(y_pred,y_test)
    ids_list = FP_TP_ids(model = model)



    f = open('TP_FP_ids_list.pkl', 'wb')
    pickle.dump(ids_list, f)
    f.close()

    model_save(model)

    # 模型特征重要性提取、展示和保存
    # feature_importance_selected(model)

    return model


def metrics_sklearn( y_pred_,y_valid):
    """模型对验证集和测试集结果的评分"""
    # 准确率
    accuracy = accuracy_score(y_valid, y_pred_)
    print('Accuracy：%.6f%%' % (accuracy * 100))

    # 精准率
    precision = precision_score(y_valid, y_pred_)
    print('Precision：%.6f%%' % (precision * 100))

    # 召回率
    recall = recall_score(y_valid, y_pred_)
    # print(y_valid)
    # print('-----------------------------')
    # print(y_pred_)
    print('Recall：%.6f%%' % (recall * 100))

    # F1值
    f1 = f1_score(y_valid, y_pred_)
    print('F1：%.6f%%' % (f1 * 100))

    # auc曲线下面积
    auc = roc_auc_score(y_valid, y_pred_)
    print('AUC：%.6f%%' % (auc * 100))

    # ks值
    fpr, tpr, thresholds = roc_curve(y_valid, y_pred_)
    ks = max(abs(fpr - tpr))
    print('KS：%.6f%%' % (ks * 100))

def model_save(clf_model):
    # 模型训练完成后做持久化，模型保存为model模式，便于调用预测
    model_path = 'xgboost_bert_model.pkl'
    pickle.dump(clf_model,open(model_path,'wb'))




    # # 模型保存为文本格式，便于分析、优化和提供可解释性
    # clf = clf_model.get_booster()
    # clf.dump_model('dump.txt')


model_xgbclf = model_fit()



Accuracy：82.818930%
Precision：13.333333%
Recall：1.282051%
F1：2.339181%
AUC：49.844457%
KS：0.311086%
